In [1]:
import streamlit as st
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

In [2]:
def case_well(X1_para,Y1_para,Q_para1,X2_para,Y2_para,Q_para2,K_para,Por_para,Qx_para ):


    H = 9.                                     # thickness [L]
    h0 = 9.5                                    # reference piezometric head [L] 
    K = K_para * 5.e-5                          # hydraulic conductivity [L/T] 
    por = Por_para                              # porosity []   old 0.25
    Qx0 = Qx_para * 1.e-10                      # baseflow in x-direction [L^2/T] was 1.e-6 before
    #Qy0 = Qy_para * 1.e-10                                    # baseflow in y-direction [L^2/T]
    Qy0 = 0
    # Wells
    xwell = np.array([X1_para, X2_para])        # x-coordinates well position [L] [99, 145]
    ywell = np.array([Y1_para, Y2_para])        # y-coordinates well position [L] [50, 78
    Qwell = np.array([Q_para1 * 1.e-4, Q_para2 * 1.e-4])   # pumping / recharge rates [L^3/T]
    R = [0.3, 0.2]                              # well radius [L]
    # Mesh
    xmin = 0           # minimum x-position of mesh [L]
    xmax = 200         # maximum x-position of mesh [L]
    ymin = 0           # minimum y-position of mesh [L]
    ymax = 200         # maximum y-position of mesh [L]

    
    # Reference point position in mesh
    iref = 1
    jref = 1

    # Graphical output options
    gsurfh = 1         # piezometric head surface plot
    gcontf = 10        # no. filled contour lines (=0: none)
    gquiv = 0          # arrow field plot
    gflowp_fit = 0     # flowpaths forward in time
    gflowp_bit = 0     # no. flowpaths backward in time (=0: none)
    gflowp_dot = 1     # flowpaths with dots indicating speed
    gstream = 10        # streamfunction plot            10



    #----------------------------------------execution-------------------------------
    xvec = np.linspace(xmin, xmax, 100)
    yvec = np.linspace(ymin, ymax, 100)
    [x, y] = np.meshgrid(xvec, yvec)                        # mesh
    phi = -Qx0 * x - Qy0 * y                                # baseflow potential
    psi = -Qx0 * y + Qy0 * x
    for i in range(0, xwell.size):                          # old version was: for i = 1:size(xwell,2)
        #r = np.sqrt((x - xwell[i]) * (x - xwell[i]) + (y - ywell[i]) * (y - ywell[i]))
        r = np.sqrt((x - xwell[i])**2 + (y - ywell[i])**2)
        phi = phi + (Qwell[i] / (2 * np.pi)) * np.log(r)    # potential
        psi = psi + (Qwell[i]/ (2 * np.pi)) * np.arctan2((y - ywell[i]), (x - xwell[i]))
    if h0 > H:
        phi0 = -phi[iref, jref] + K * H * h0 - 0.5 * K * H * H 
    else:
        phi0 = -phi[iref, jref] + 0.5 * K * h0 * h0          # reference potential                                                 
    hc = 0.5 * H + (1 / K / H) * (phi + phi0)                     # head confined
    hu = np.sqrt((2 / K) * (phi + phi0))                      # head unconfined
    phicrit = phi0 + 0.5 * K * H * H                        # transition confined / unconfined
    confined = (phi >= phicrit)                         # confined / unconfined indicator
    h = confined * hc+ ~confined * hu                    # head

    [u,v] = np.gradient(-phi)                           # discharge vector  
    Hh = confined * H + ~confined * h                   # aquifer depth  
    u = u / Hh / (xvec[2] - xvec[1]) / por
    v = v / Hh / (yvec[2] - yvec[1]) / por
    #--------------------------------------graphical output--------------------
    if gsurfh: 
        #plt.figure()
        fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
        surf = ax.plot_surface(x, y, h,cmap=cm.coolwarm,linewidth=0,antialiased=True)                         # surface 
        #plt.gca().zaxis.set_major_formatter(StrMethodFormatter('{x:,.4f}'))
        ax.set_xlabel('x [m]')
        ax.set_ylabel('y [m]')

        ax.set_zlabel('drawdown [m]')
        fig.colorbar(surf, shrink=.8, ax=[ax], location = "left") # ax=[ax], location='left' for left side

        #fig.colorbar(surf, shrink=0.5, aspect=10)
    if gcontf or gquiv or gflowp_fit or gflowp_bit or gflowp_dot or gstream:
        fig2, ax = plt.subplots()

        contour = plt.contour(x, y, h, gcontf, cmap=cm.Blues)
        contour2 = plt.contour(x, y, psi, gstream, colors=['#808080', '#808080', '#808080'], extend='both')

        ax.set_xlabel('x [m]')
        ax.set_ylabel('y [m]')

        contour.cmap.set_over('#808080')
        contour.cmap.set_under('blue')
        contour.changed()

        # Manually add color patches to the legend
        labels = ['Streamline', 'Potentialline']
        handles = [
            plt.Line2D([0], [0], color='#808080', lw=2),
            plt.Line2D([0], [0], color='blue', lw=2),
        ]

        plt.legend(handles, labels, loc='upper left')



    if gquiv:
        plt.quiver(x,y,v,u)                          # arrow field // quiver(x,y,u,v,'y') 
    if gflowp_fit:                                      # flowpaths 
        xstart = []
        ystart = []
        for i in range(100):
            if v[1,i] > 0:
                xstart = [xstart, xvec[i]]
                ystart = [ystart, yvec[1]]
            if v[99,i] < 0:
                xstart = [xstart, xvec[i]]
                ystart = [ystart, yvec[99]]
            if u[i,1] > 0:
                xstart = [xstart, xvec[1]]
                ystart = [ystart, yvec[i]]
            if u[i,99] < 0:
                xstart = [xstart, xvec[99]]
                ystart = [ystart, yvec[i]]
    
        h = plt.streamplot(x,y,u,v,)#,xstart,ystart)
        plt.streamplot(x,y,u,v,color='b')#,xstart,ystart)
  
    if gflowp_bit:          
        for j in range(0, Qwell.size):
            if Qwell[j]>0:           # only for pumping wells
                xstart = xwell[j] + R[j]*np.cos(2*np.pi*np.array([1,1,gflowp_bit])/gflowp_bit) 
                ystart = ywell[j] + R[j]*np.sin(2*np.pi*np.array([1,1,gflowp_bit])/gflowp_bit)
                seed_points = np.array([xstart,ystart])

                h = plt.streamplot(x,y,-u,-v,start_points=seed_points.T)


    
    
    plt.show()
#Testing the code
#case_well(X1_para=15,Y1_para=25,Q_para1=2,X2_para=30,Y2_para=40,Q_para2=1,K_para=2.1,Por_para=0.6,Qx_para=2 )

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Function to create and display widgets based on the number of wells selected
def create_widgets(num_wells):
    #print(num_wells)
    # Define the widgets
    style = {'description_width': 'initial'}
    if num_wells==2 : 
        X1_para = widgets.FloatText(value=75, description='X Position Well 1',style=style)
        Y1_para = widgets.FloatText(value=100, description='Y Position Well 1',style=style)
        Q_para1 = widgets.FloatText(value=1, description='Pumping Rate Well 1',style=style)
        X2_para = widgets.FloatText(value=150, description='X Position Well 2',style=style)
        Y2_para = widgets.FloatText(value=100, description='Y Position Well 2',style=style)
        Q_para2 = widgets.FloatText(value=1, description='Pumping Rate Well 2',style=style)
        K_para = widgets.FloatText(value=6.3, description='Hydraulic Conductivity (K)',style=style)
        Por_para = widgets.FloatText(value=0.6, description='Porosity (ne)',style=style)
        Qx_para = widgets.FloatText(value=1, description='Baseflow Discharge (Qx)',style=style)
        
        # Function to be called on widget change
        def on_widget_change(change):
            clear_output(wait=True)
            hbox1 = widgets.HBox([X1_para, Y1_para, Q_para1])
            hbox2 = widgets.HBox([X2_para, Y2_para, Q_para2])
            hbox3 = widgets.HBox([K_para, Por_para, Qx_para])
            vbox = widgets.VBox([hbox1, hbox2, hbox3])
            display(vbox)
            case_well(
                X1_para.value, Y1_para.value, Q_para1.value, 
                X2_para.value, Y2_para.value, Q_para2.value, 
                K_para.value, Por_para.value, Qx_para.value
            )

        # Observe changes in the widgets
        X1_para.observe(on_widget_change, 'value')
        Y1_para.observe(on_widget_change, 'value')
        Q_para1.observe(on_widget_change, 'value')
        X2_para.observe(on_widget_change, 'value')
        Y2_para.observe(on_widget_change, 'value')
        Q_para2.observe(on_widget_change, 'value')
        K_para.observe(on_widget_change, 'value')
        Por_para.observe(on_widget_change, 'value')
        Qx_para.observe(on_widget_change, 'value')

        # Initial display
        on_widget_change(None)
    if num_wells==1 : 
        X1_para = widgets.FloatText(value=75, description='X Position Well 1',style=style)
        Y1_para = widgets.FloatText(value=100,  description='Y Position well 1',style=style)
        Q_para1 = widgets.FloatText(value=1,  description='Pumping Rate Well 1',style=style)
        #X2_para = widgets.FloatText(value=150, description='X-well 2')
        #Y2_para = widgets.FloatText(value=100, description='Y-well 2')
        #Q_para2 = widgets.FloatText(value=1, description='Pumping 2')
        K_para = widgets.FloatText(value=6.3,description='Hydraulic Conductivity (K)',style=style)
        Por_para = widgets.FloatText(value=0.6, description='Porosity (ne)',style=style)
        Qx_para = widgets.FloatText(value=1, description='Baseflow Discharge (Qx)',style=style)
        
        # Function to be called on widget change
        def on_widget_change(change):
            clear_output(wait=True)
            hbox1 = widgets.HBox([X1_para, Y1_para, Q_para1])
            #hbox2 = widgets.HBox([X2_para, Y2_para, Q_para2])
            hbox3 = widgets.HBox([K_para, Por_para, Qx_para])
            vbox = widgets.VBox([hbox1, hbox3])
            X2_para=0
            Y2_para=0
            Q_para2=0
            display(vbox)
            case_well(
                X1_para.value, Y1_para.value, Q_para1.value, 
                X2_para, Y2_para, Q_para2, 
                K_para.value, Por_para.value, Qx_para.value
            )

        # Observe changes in the widgets
        X1_para.observe(on_widget_change, 'value')
        Y1_para.observe(on_widget_change, 'value')
        Q_para1.observe(on_widget_change, 'value')
        #X2_para.observe(on_widget_change, 'value')
        #Y2_para.observe(on_widget_change, 'value')
        #Q_para2.observe(on_widget_change, 'value')
        K_para.observe(on_widget_change, 'value')
        Por_para.observe(on_widget_change, 'value')
        Qx_para.observe(on_widget_change, 'value')

        # Initial display
        on_widget_change(None)

# Dropdown to select the number of wells
num_wells_dropdown = widgets.Dropdown(options=['Select',1, 2], description='Number of Wells')

# Function to be called on dropdown change
def on_dropdown_change(change):
    clear_output(wait=True)
    display(num_wells_dropdown)
    create_widgets(change.new)

# Observe changes in the dropdown
num_wells_dropdown.observe(on_dropdown_change, 'value')

# Display the dropdown
display(num_wells_dropdown)

Dropdown(description='Number of Wells', options=('Select', 1, 2), value='Select')